In [1]:
import torch

def print_model_parm_nums(model):
    """
    Used for calculate models' parameter number.
    :param model:
    :return:
    """
    total = sum([param.nelement() for param in model.parameters()])
    print('  + Number of params: %.2fM' % (total / 1e6))
    return '  + Number of params: %.2fM' % (total / 1e6)


def print_disc_model_flops(model, inp_h=32, inp_w=32, multiply_adds=False):
    """
    Used for calculate the FLOPs of discriminative model.
    :param model: nn.Module object.
    :param inp_h: The height of the input image.
    :param inp_w:The width of the input image.
    :param multiply_adds: enable multiply_adds or not.
    :return:
    """
    print('Using input shape: %d * %d * 3' % (inp_h, inp_w))
    list_conv = []
    list_linear = []
    list_bn = []
    list_relu = []
    list_pooling = []

    def conv_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()

        kernel_ops = self.kernel_size[0] * self.kernel_size[1] * (self.in_channels / self.groups) * (
            2 if multiply_adds else 1)
        bias_ops = 1 if self.bias is not None else 0

        params = output_channels * (kernel_ops + bias_ops)
        flops = batch_size * params * output_height * output_width

        list_conv.append(flops)

    def linear_hook(self, input, output):
        batch_size = input[0].size(0) if input[0].dim() == 2 else 1

        weight_ops = self.weight.nelement() * (2 if multiply_adds else 1)
        bias_ops = self.bias.nelement()

        flops = batch_size * (weight_ops + bias_ops)
        list_linear.append(flops)

    def bn_hook(self, input, output):
        list_bn.append(input[0].nelement())

    def relu_hook(self, input, output):
        list_relu.append(input[0].nelement())

    def pooling_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()

        kernel_ops = self.kernel_size * self.kernel_size
        bias_ops = 0
        params = output_channels * (kernel_ops + bias_ops)
        flops = batch_size * params * output_height * output_width

        list_pooling.append(flops)

    def adaptive_pooling_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()

        if output_height == 1 and output_width == 1:
            kernel_ops = input_height * input_width
        else:
            raise NotImplementedError

        bias_ops = 0
        params = output_channels * (kernel_ops + bias_ops)
        flops = batch_size * params * output_height * output_width

        list_pooling.append(flops)

    def foo(net):
        childrens = list(net.children())
        if not childrens:
            if isinstance(net, torch.nn.Conv2d):
                net.register_forward_hook(conv_hook)
            if isinstance(net, torch.nn.Linear):
                net.register_forward_hook(linear_hook)
            if isinstance(net, torch.nn.BatchNorm2d):
                net.register_forward_hook(bn_hook)
            if isinstance(net, torch.nn.ReLU) or isinstance(net, torch.nn.Sigmoid) or isinstance(net, torch.nn.PReLU):
                net.register_forward_hook(relu_hook)
            if isinstance(net, torch.nn.MaxPool2d) or isinstance(net, torch.nn.AvgPool2d):
                net.register_forward_hook(pooling_hook)
            if isinstance(net, torch.nn.AdaptiveAvgPool2d):
                net.register_forward_hook(adaptive_pooling_hook)
            return
        for c in childrens:
            foo(c)

    foo(model)
    input = Variable(torch.rand(3, inp_h, inp_w).unsqueeze(0), requires_grad=False)
    with torch.no_grad():
        out = model(input)
    del out
    total_flops = (sum(list_conv) + sum(list_linear) + sum(list_bn) + sum(list_relu) + sum(list_pooling))

    print('  + Number of FLOPs: %.2fG' % (total_flops / 1e9))

In [2]:
import argparse
import os
import time
import logging
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import models
from data import get_dataset
from preprocess import get_transform
from utils.log import setup_logging, ResultsLog, save_checkpoint
from utils.meters import AverageMeter, accuracy
from utils.optim import OptimRegime
from utils.misc import torch_dtypes
from datetime import datetime
from ast import literal_eval
import numpy as np
from functools import reduce

In [15]:
model = models.__dict__['resnet']

In [16]:
model_config = {'input_size': None, 'dataset': 'cifar10', 'depth':50}

In [17]:
ResNet50 = model(**model_config)

In [6]:
model

ResNet_cifar10(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    

In [9]:
print_disc_model_flops(model, inp_h=32, inp_w=32, multiply_adds=True)

Using input shape: 32 * 32 * 3
  + Number of FLOPs: 0.05G


In [8]:
from torch.autograd import Variable

In [10]:
model_config = {'input_size': None, 'dataset': 'cifar10', 'depth':34}

In [13]:
ResNet34 = model(**model_config)

In [14]:
print_disc_model_flops(ResNet34, inp_h=32, inp_w=32, multiply_adds=True)

Using input shape: 32 * 32 * 3
  + Number of FLOPs: 0.14G


In [18]:
print_disc_model_flops(ResNet50, inp_h=32, inp_w=32, multiply_adds=True)

Using input shape: 32 * 32 * 3
  + Number of FLOPs: 0.22G


In [2]:
print('layer{}'.format(1.2))

layer1.2


In [3]:
2/3

0.6666666666666666